In [ ]:
J_arr = np.arange(0,2.1,0.4)
lambda_arr= np.arange(0,0.5,0.04)

In [ ]:
set_random_seed(42)
for i in range(6,len(lambda_arr)):
    
    phi_4 = Phi4(N_nod,Beta,Space_dim,Mass,L_nod,R,0,lambda_arr[i])
    pipeline = Pipeline(model=rg, 
                  latent = normal_dist ,
                  criterion = phi_4.get_KL(), 
                  optimizer_class=torch.optim.Adam,
                  optimizer_kwargs={"lr": 0.001,"weight_decay":0.0})

    trainer = pl.Trainer(
        max_epochs = 10,
        logger = TensorBoardLogger(save_dir=f"./logs/field_phi4_lambda"),
        num_sanity_val_steps = 0,
        log_every_n_steps = 1,
        enable_checkpointing = False,
        accumulate_grad_batches = 1)

    trainer.fit(model=pipeline, train_dataloaders=train_loader)
    rg.save("./weights_phi4_lambda/model_weights_phi4_lambda"+str(lambda_arr[i])+".pth")

In [ ]:
hidden_dim_dict1={8:256}
n_flows_dict1={8:4}
num_hidden_dict1={8:4}
set_random_seed(42)
rg = RGflows.configure_RG_model(RGmasks,n_flows_dict1,num_hidden_dict1,hidden_dim_dict1,grids_no_grad=0,p_drop=0.0,sys_dim = scalar.dim, O_latt = O_latt)

In [ ]:
for i in range(14):
    
    phi_4 = Phi4(N_nod,Beta,Space_dim,Mass,L_nod,R,0,0.48)
    pipeline = Pipeline(model=rg, 
                  latent = normal_dist ,
                  criterion = phi_4.get_KL(), 
                  optimizer_class=torch.optim.Adam,
                  optimizer_kwargs={"lr": 0.001,"weight_decay":0.0})

    trainer = pl.Trainer(
        max_epochs = 15,
        logger = TensorBoardLogger(save_dir=f"./logs/field_phi4_numberaff"),
        num_sanity_val_steps = 0,
        log_every_n_steps = 1,
        enable_checkpointing = False,
        accumulate_grad_batches = 1)
    rg.append_aff(256,6,2)
    trainer.fit(model=pipeline, train_dataloaders=train_loader)
    rg.save("./weights_phi4_number_aff/model_weights_phi4 "+str(4+2*i)+".pth")

In [ ]:
set_random_seed(42)
for i in range(len(J_arr)):
    
    phi4.set_J_global(J_arr[i])
    pipeline = Pipeline(model=rg, 
                  latent = normal_dist ,
                  criterion = phi4.get_KL(), 
                  optimizer_class=torch.optim.Adam,
                  optimizer_kwargs={"lr": 0.001,"weight_decay":0.0})

    trainer = pl.Trainer(
        max_epochs = 10,
        logger = TensorBoardLogger(save_dir=f"./logs/field_phi4_J_global"),
        num_sanity_val_steps = 0,
        log_every_n_steps = 1,
        enable_checkpointing = False,
        accumulate_grad_batches = 1)

    trainer.fit(model=pipeline, train_dataloaders=train_loader)
    rg.save("./weights/field/phi4_J/phi_4_J"+str(J_arr[i])+".pth")

In [ ]:
set_random_seed(42)
phi4.set_J_local(1,4)
phi4.mass2 = 0.0625
for i in range(len(beta_arr)):
    
    phi4.beta = beta_arr[i]
    pipeline = Pipeline(model = rg, 
                  latent = normal_dist ,
                  criterion = phi4.get_KL(), 
                  optimizer_class=torch.optim.Adam,
                  optimizer_kwargs={"lr": 0.001,"weight_decay":0.0})

    trainer = pl.Trainer(
        max_epochs = 15,
        logger = TensorBoardLogger(save_dir=f"./logs/field_phi4_beta"),
        num_sanity_val_steps = 0,
        log_every_n_steps = 1,
        enable_checkpointing = False,
        accumulate_grad_batches = 1)

    trainer.fit(model=pipeline, train_dataloaders=train_loader)
    rg.save("./weights/field/phi4_beta_jloc/phi4_beta_jloc"+str(beta_arr[i])+".pth")

In [ ]:
J_arr=np.array([0,0.2,0.5,0.7,1.0,1.2,1.4,1.6,1.8,2.0])

In [ ]:
set_random_seed(42)
J_arr=np.array([0,0.2,0.5,0.7,1.0,1.2,1.4,1.6,1.8,2.0])

for i in range(len(J_arr)):
     
    phi_4 = Phi4(latt,1,1/24)
    phi_4.set_J_global(torch.tensor(J_arr[i]))

    pipeline = Pipeline(model = nf, 
                  latent = normal_dist ,
                  criterion = phi_4.get_KL(), 
                  optimizer_class=torch.optim.Adam,
                  optimizer_kwargs={"lr": 0.001,"weight_decay":0.1}).to(device)

    trainer = pl.Trainer(
        max_epochs = 10,
        logger = TensorBoardLogger(save_dir=f"./logs/field"),
        num_sanity_val_steps = 0,
        log_every_n_steps = 1,
        enable_checkpointing = False,
        accumulate_grad_batches = 1)
    
   
    trainer.fit(model=pipeline, train_dataloaders = train_loader)
    nf.save("./weights/field_glob"+str(J_arr[i])+".pth")

In [ ]:
from samplers.Langevin import Langevin
from samplers.NN_Metropolis import NN_Metropolis

for i in range(len(J_arr)):
    x = normal_dist.sample((1,)).to(device)
     

    nf = NormalizingFlow.load_model("./weights/field_glob"+str(J_arr[i])+".pth").to(device)
    nf.eval()


    phi_4 = Phi4(latt,1,1/24)
    phi_4.set_J_global(torch.tensor(J_arr[i]))

    N_samp=1000
    Lang = Langevin(phi_4,1,val = "none",eps = 0.001,N_sweep = 100,log_per = 1000,filename = "./trajs_and_corr/1.txt",open_mode = "w")
    nn_M = NN_Metropolis(phi_4,N_samp,nf,Lang,log_per = 1000,filename="trajs_and_corr/globalJ"+str(J_arr[i])+".txt",stat_filename="./trajs_and_corr/stat_global"+str(J_arr[i])+".txt")
    nn_M.run(x)

In [ ]:
G=np.array([1,2,4,8])

In [ ]:
from samplers.Langevin import Langevin
from samplers.NN_Metropolis import NN_Metropolis

G=np.array([1,2,4,8])
for i in range(len(G)):
    x = normal_dist.sample((1,)).to(device)
     

    nf = NormalizingFlow.load_model("./weights/field"+str(G[i])+".pth").to(device)
    nf.eval()


    phi_4 = Phi4(latt,1,G[i]/24)
    phi_4.set_J_local(1,[0,L//2])


    N_samp=1000
    Lang = Langevin(phi_4,1,val = "none",eps = 0.001,N_sweep = 100,log_per = 1000,filename = "./trajs_and_corr/1.txt",open_mode = "w")
    nn_M = NN_Metropolis(phi_4,N_samp,nf,Lang,log_per = 1000,filename="trajs_and_corr/localJ"+str(G[i])+".txt",stat_filename="./trajs_and_corr/stat"+str(G[i])+".txt")
    nn_M.run(x)

In [ ]:
plt.figure()
for i in range(4):
    
    filename="trajs_and_corr/localJ"+str(G[i])+".txt"
    
    phi = np.genfromtxt(filename,delimiter=" ")
    phi = torch.tensor(phi).to(device).to(torch.float32)

    err = torch.std(phi,0)/(1000**0.5)
    phi = torch.mean(phi,0)

    av = sf.lattice.get_time_averaging_mat()
    phi = torch.matmul(av,phi)
    err = torch.matmul(av,err)


    x = latt.steps[1]*np.arange(latt.n_nodes[1])
    plt.scatter(x,-phi.cpu())
    plt.errorbar(x,-phi.cpu(),err.cpu(),linestyle="none")
plt.xlabel("x",fontsize=17)
plt.ylabel(r"$|\langle \phi \rangle|$",fontsize=17)    
plt.legend([r"$\lambda=1$",r"$\lambda=2$",r"$\lambda=4$",r"$\lambda=8$"])
plt.title(r"$J=\delta(x-2)$",fontsize=17)
plt.grid(True)   
plt.savefig("./localJ1.pdf") 
plt.show()

In [ ]:
n_exp = len(J_arr)
mean = np.zeros(n_exp)
err = np.zeros(n_exp)
for i in range(n_exp):
    
    filename="trajs_and_corr/globalJ"+str(J_arr[i])+".txt"
    
    phi = np.genfromtxt(filename,delimiter=" ")
    phi = torch.tensor(phi).to(device).to(torch.float32)
    
    err[i] = torch.std(phi)/((1000 * 256)**0.5)
    mean[i] = torch.mean(phi)


plt.figure()
plt.scatter(J_arr,-mean)
plt.errorbar(J_arr,-mean,err)
plt.plot(J_arr,J_arr)
plt.xlabel("J",fontsize=17)
plt.ylabel(r"$|\langle \phi \rangle|$",fontsize=17)    
plt.legend([r"$\lambda=1$",r"$\lambda=0$"])
#plt.title(r"",fontsize=17)
plt.grid(True)   
plt.savefig("./globalJ.pdf") 
plt.show()